## Models for all states with various feature combos

In [53]:
import pandas as pd

In [54]:
better_col_names = {
    'xgbClassifier': 'Simple Classification',
    'xgbOrdinalClassifier': 'Ordinal Classification',
    'xgbCoarseGrainedClassifier': 'Binned(3 Bins) Classification'
}

better_index_name = {
    'babelkw': 'Babelfy KWs',
    'kw': 'YAKE KWs',
    'wikikw': 'Wikifier KWs',
    'st': 'Sentence Transformer',
    'tfidf': 'TF-IDF'
}


In [55]:
results_dir = '../results/results_with_combos/accuracies'
parsed = dict()
with open(results_dir) as f:
    for l in f:
        label, acc = l.split('\t')
        parsed[label] = acc

In [56]:
results = []
for k in parsed:
    model, features, filtered = k.split('.')[0].split('_')
    filtered = filtered == 'True'
    accuracy = float(parsed[k].strip())
    results.append((model, features, filtered, accuracy))
df = pd.DataFrame(results, columns=['model', 'features', 'filtered', 'accuracy'])

In [57]:
df[df.filtered].\
    drop(columns=['filtered']).\
    pivot(index='model', columns='features').\
    transpose().\
    rename(columns=better_col_names, index=better_index_name).\
    style.highlight_max(color='lightgreen').highlight_min(color='red')

In [59]:
df[~df.filtered].\
    drop(columns=['filtered']).\
    pivot(index='model', columns='features').\
    transpose().\
    rename(columns=better_col_names, index=better_index_name).\
    style.highlight_max(color='lightgreen').highlight_min(color='red')

## Models for each state

In [65]:
results_dir = '../results/results_states/accuracies'
parsed = dict()
with open(results_dir) as f:
    for l in f:
        label, acc = l.split('\t')
        parsed[label] = acc

In [73]:
results = []
for k in parsed:
    model, features, filtered, state = k.split('.')[0].split('_')
    filtered = filtered == 'True'
    accuracy = float(parsed[k].strip())
    results.append((model, features, filtered, state, accuracy))
df = pd.DataFrame(results, columns=['model', 'features', 'filtered', 'state', 'accuracy'])

In [74]:
df[~df.filtered].\
    drop(columns=['filtered', 'model']).\
    pivot(index='state', columns='features').\
    transpose().\
    rename(columns=better_col_names, index=better_index_name).\
    style.highlight_max(color='lightgreen').highlight_min(color='red')